# Aufgaben zu Big Data File Formats

Folgende Aufgaben haben zum Ziel mit den verschiedenen Dateiformaten vertraut zu werden und insbesondere die speziellen Eigenschaften und Funktionen der Formate zu verstehen

### CSV and JSON
Klassische Datei Formate für Datenverarbeitung  
**Typische Eigenschaften:** Einfache Struktur, human-readable, Zeilenformat

### Avro, ORC, Parquet
Big Data optimierte Formate um schnell große Datenmengen zu lesen und zu schreiben  
**Typische Eigenschaften:** teilbar in kleine Dateien (splittable), komprimierbar (compressible), überspringbar (skippable), selbsterklärend (self describing with schema), Schema erweiterbar (Schema Evolution), Schema erzwingend (Schema Enforcment), Filter Pushdown

### Delta, Iceberg, Hudi
Erweiterte Big Data Formate um die ACID und Tracing Eigenschaften einer klassichen SQL Datenbank zu erfüllen  
**Typische Eigenschaften:** Erweiterung um zusätzliche Metadaten und spezielle Treiber zum lesen/schreiben, Time Travel Funktion, Merge und Update Funktionen, Audit Log Funktionalitäten

###  Import Python Modules
Hier werden alle benötigten Libraries für dieses Lab heruntergeladen.


In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as f

from delta import *

import datetime
from datetime import datetime, timedelta

import json
import csv

import boto3
from botocore.client import Config

# use 95% of the screen for jupyter cell
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

### Launch Spark Jupyter and Configuration

#### Configure a Spark session for Kubernetes cluster with S3 support
### CLUSTER MANAGER
- set the Kubernetes master URL as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type)

### KUBERNETES
- set the namespace that will be used for running the driver and executor pods
- set the docker image from which the Worker/Exectutor pods are created
- set the Kubernetes service account name and provide the authentication details for the service account (required to create worker pods)

### SPARK
- set the driver host and the driver port (find name of the driver service with 'kubectl get services' or in the helm chart configuration)
- enable Delta Lake, Iceberg, and Hudi support by setting the spark.sql.extensions
- configure Hive catalog for Iceberg
- enable S3 connector
- set the number of worker pods, their memory and cores (HINT: number of possible tasks = cores * executores)

### SPARK SESSION
- create the Spark session using the SparkSession.builder object
- get the Spark context from the created session and set the log level to "ERROR".


In [ ]:
# spark.stop()

In [2]:
appName="jupyter-spark"

conf = SparkConf()

# CLUSTER MANAGER

conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")

# CONFIGURE KUBERNETES

conf.set("spark.kubernetes.namespace","frontend")
conf.set("spark.kubernetes.container.image", "thinkportgmbh/workshops:spark-3.3.2")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
conf.set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")

# CONFIGURE SPARK

conf.set("spark.sql.session.timeZone", "Europe/Berlin")
conf.set("spark.driver.host", "jupyter-spark-driver.frontend.svc.cluster.local")
conf.set("spark.driver.port", "29413")

conf.set("spark.jars", "/opt/spark/jars/spark-avro_2.12-3.3.2.jar")
conf.set("spark.driver.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.2.jar")
conf.set("spark.executor.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.2.jar")

conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions, org.apache.spark.sql.hudi.HoodieSparkSessionExtension")

######## Hive als Metastore einbinden
#conf.set("hive.metastore.uris", "thrift://hive-metastore.hive.svc.cluster.local:9083") 

######## Iceberg configs
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
conf.set("spark.sql.catalog.ice","org.apache.iceberg.spark.SparkCatalog") 
conf.set("spark.sql.catalog.ice.type","hive") 
conf.set("spark.sql.catalog.ice.uri","thrift://hive-metastore.hive.svc.cluster.local:9083") 

####### Hudi configs
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# CONFIGURE S3 CONNECTOR
conf.set("spark.hadoop.fs.s3a.endpoint", "minio.minio.svc.cluster.local:9000")
conf.set("spark.hadoop.fs.s3a.access.key", "trainadm")
conf.set("spark.hadoop.fs.s3a.secret.key", "train@thinkport")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

# CONFIGURE WORKER (Customize based on workload)

conf.set("spark.executor.instances", "1")
conf.set("spark.executor.memory", "1G")
conf.set("spark.executor.cores", "2")

# SPARK SESSION

spark = SparkSession\
    .builder\
    .config(conf=conf) \
    .config('spark.sql.session.timeZone', 'Europe/Berlin') \
    .appName(appName)\
    .enableHiveSupport() \
    .getOrCreate()


sc=spark.sparkContext
sc.setLogLevel("ERROR")

# get the configuration object to check all the configurations the session was startet with
for entry in sc.getConf().getAll():
        if entry[0] in ["spark.app.name","spark.kubernetes.namespace","spark.executor.memory","spark.executor.cores","spark.driver.host","spark.master","spark.sql.extensions"]:
            print(entry[0],"=",entry[1])

23/06/29 13:13:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


spark.kubernetes.namespace = frontend
spark.master = k8s://https://kubernetes.default.svc.cluster.local:443
spark.app.name = jupyter-spark
spark.executor.memory = 1G
spark.executor.cores = 2
spark.sql.extensions = org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
spark.driver.host = jupyter-spark-driver.frontend.svc.cluster.local


### Configure Boto3 
for simple s3 operations

In [ ]:
# Hilfsfunktionen um mit einfachen Befehlen auf s3 zu arbeiten 
# WICHTIG: Falls das bucket s3://fileformats noch nicht existiert muss dieses über das Terminal erst erzeugt werden
# Command: s3 mb s3://fileformats

# Bucket, muss zuerst in Minio oder via Terminal Befehl erstellt werden
bucket = "fileformats"
bucket_path="s3://"+bucket

options = {
    'endpoint_url': 'http://minio.minio.svc.cluster.local:9000',
    'aws_access_key_id': 'trainadm',
    'aws_secret_access_key': 'train@thinkport',
    'config': Config(signature_version='s3v4'),
    'verify': False}

s3_resource = boto3.resource('s3', **options)  
s3_client = boto3.client('s3', **options)

# show files on s3 bucket/prefix
def ls(bucket,prefix):
    '''List objects from bucket/prefix'''
    try:
        for obj in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
            print(obj.key)
    except Exception as e: 
        print(e)
    
# show file content in files
def cat(bucket,prefix,binary=False):
    '''Show content of one or several files with same prefix/wildcard'''
    try:
        for obj in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
            print("File:",obj.key)
            print("----------------------")
            if binary==True:
                print(obj.get()['Body'].read())
            else: 
                print(obj.get()['Body'].read().decode())
            print("######################")
    except Exception as e: 
        print(e)

# delete files from bucket
def rm(bucket,prefix):
    '''Delete everything from bucket/prefix'''
    for object in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix):
        print(object.key)
        s3_client.delete_object(Bucket=bucket, Key=object.key)
    print(f"Deleted files from {bucket}/{prefix}*")


In [ ]:
# show everything in bucket
ls(bucket,"")
print("#############################")
# show folder
ls(bucket,"csv")
print("#############################")
# show subfolder
ls(bucket,"delta/_delta_log/")
print("#############################")
print("")
# show content of one or several files with same prefix/wildcard
cat(bucket,'csv/part')

### Create sample data

In [8]:
# initial Daten
account_data1 = [
    (1,"alex","2019-01-01",1000),
    (2,"alex","2019-02-01",1500),
    (3,"alex","2019-03-01",1700),
    (4,"maria","2020-01-01",5000)
    ]

# Datensatz mit einem Update und einer neuen Zeile
account_data2 = [
    (1,"alex","2019-03-01",3300),
    (2,"peter","2021-01-01",100)
    ]

# Datensatz mit neuer Zeile und neuer Spalte
account_data3 = [
    (1,"otto","2019-10-01",4444,"neue Spalte 1")
]

# Datensatz mit neuer Zeile und neuer Spalte
account_data4 = [
    (5,"markus","2019-09-01",555)
]

schema = ["id","account","dt_transaction","balance"]
schema3 = ["id","account","dt_transaction","balance","new"]

df1 = spark.createDataFrame(data=account_data1, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(3)
df2 = spark.createDataFrame(data=account_data2, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(2)
df3 = spark.createDataFrame(data=account_data3, schema = schema3).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).repartition(1)
df4 = spark.createDataFrame(data=account_data4, schema = schema).withColumn("dt_transaction",f.col("dt_transaction").cast("date")).withColumn("id",f.col("id").cast("string")).repartition(1)


print("++ create new dataframe and show schema and data")
print("################################################")

# df1.printSchema()
print("++ start data")
df1.show(truncate=False)
print("++ update row and add row")
df2.show(truncate=False)
print("++ add new column")
df3.show(truncate=False)
print("++ add new row with wrong schema (id)")
df4.show(truncate=False)
df1.printSchema()
df4.printSchema()

++ create new dataframe and show schema and data
################################################
++ start data


+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|1  |alex   |2019-01-01    |1000   |
|2  |alex   |2019-02-01    |1500   |
|4  |maria  |2020-01-01    |5000   |
|3  |alex   |2019-03-01    |1700   |
+---+-------+--------------+-------+

++ update row and add row
+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|1  |alex   |2019-03-01    |3300   |
|2  |peter  |2021-01-01    |100    |
+---+-------+--------------+-------+

++ add new column
+---+-------+--------------+-------+-------------+
|id |account|dt_transaction|balance|new          |
+---+-------+--------------+-------+-------------+
|1  |otto   |2019-10-01    |4444   |neue Spalte 1|
+---+-------+--------------+-------+-------------+

++ add new row with wrong schema (id)
+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|5  |markus |2019-09-01    |555

<hr style="height: 3px; background: gray;">

## CSV

### Aufgabe:
Schreibe die Daten als CSV mit der Overwrite und Append Funktion

1. Datenset 1 als csv schreiben (.format("csv") und Pfad= .save(f"s3://{bucket}/csv"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen erhalten wurden
4. Datenset 3 anfügen mit weiterer Spalte anfügen (append)
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert


In [ ]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

# Schreibe Datenset 1 als CSV Datei
write_csv=(df1
           .write
           .format("csv")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/csv")
          )


In [ ]:
# Anzeigen der Dateien im Bucket/Prefix
ls(bucket,"csv")

In [ ]:
# Anzeigen der Inhalte jeder Datei im Bucket/Prefix
cat(bucket,"csv/part")

In [ ]:
# lese die csv Datei wieder ein und prüfe das Schema
read_csv=spark.read.format("csv").load(f"s3://{bucket}/csv")

read_csv.printSchema()
read_csv.show()

In [ ]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu
write_csv=(df3
           .write
           .format("csv")
           .mode("append")
           .save(f"s3://{bucket}/csv")
          )

In [ ]:
# Anzeigen der Inhalte jeder Datei im Bucket/Prefix
ls(bucket,"csv")

In [ ]:
# Anzeigen der Inhalte jeder Datei im Bucket/Prefix
cat(bucket,"csv/part")

In [ ]:
# und lese alles nochmal ein um zu schauen ob die neue Spalte richtig erkannt wurde
read_csv=spark.read.format("csv").load(f"s3://{bucket}/csv")

read_csv.printSchema()
read_csv.show()

#### Erkenntnisse CSV
* In wieviele Dateien wird das Datenset aufgeteilt und warum?
* Bleibt das Schema erhalten (Selbsterklärend)
* Können neue Spalten angefügt werden (Schema Evolution)

<hr style="height: 3px; background: gray;">

## JSON

### Aufgabe:
Wiederhole die gleichen Schritte mit dem JSON Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als json schreiben (.format("json") und Pfad= .save(f"s3://{bucket}/json"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Datenset 3 anfügen (append)
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert


In [ ]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

# Schreibe Datenset 1 als JSON Datei

# HIER EIGENE CODE EINFÜGEN - in den pfad s3://{bucket}/json schreiben

<details>
<summary> &#8964 Lösung Datenset 1 als JSON Datei schreiben</summary>
<p>
<code>write_json=(df1
           .write
           .format("json")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/json")
          )</code>
</details>
</p>

In [ ]:
ls(bucket,"json")

In [ ]:
cat(bucket,"json/part")

In [ ]:
# Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt

# HIER EIGENE CODE EINFÜGEN

In [ ]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu (!! append NOT overwrite)

# HIER EIGENE CODE EINFÜGEN

<details>
<summary> &#8964 Lösung Datenset 3 als JSON Datei anfügen</summary>
<p>
<code>write_json=(df3
           .write
           .format("json")
           .mode("append") # append
           .save(f"s3://{bucket}/json")
          )</code>
</details>
</p>

In [ ]:
# alles nochmal einlesen und schauen ob die neue Spalte und die Schemas richtig erkannt wurden

read_json=# HIER EIGENE CODE EINFÜGEN

read_json.printSchema()
read_json.show()

<details>
<summary> &#8964 Lösung JSON wieder einlesen</summary>
<p>
<code>
read_json=spark.read.format("json").load(f"s3://{bucket}/json")
read_json.printSchema()
read_json.show()
    </code>
</details>
</p>

#### Erkenntnisse JSON
* Bleibt das Schema erhalten (Selbsterklärend)
* Können neue Spalten angefügt werden (Schema Evolution)

## AVRO
Avro ist ein Zeilenformat was für das schnelle Schreiben im Streaming Kontext optimiert ist.
Avro ist selbsterklärend, hat ein Schema und unterstützt Schema Evolution

### Aufgabe:
Wiederhole die gleichen Schritte mit dem AVRO Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als avro schreiben (.format("avro") und Pfad= .save(f"s3://{bucket}/avro"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Metadaten in Datei identifizieren
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Schema Evolutiuon: Datenset 3 anfügen mit neuer Spalte anfügen
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert
6. Schema Enforcement: Datentyp in bestehender Spalte ändern und schauen ob und wie dies gehandhabt wird

In [ ]:
# Schreibe Datenset 1 als AVRO Datei

# <<HIER EIGENE CODE EINFÜGEN>> - in den Pfad s3://{bucket}/avro schreiben


<details>
<summary> &#8964 Lösung Datenset 1 als AVRO Datei schreiben</summary>
<p>
<code>write_avro=(df1
           .write
           .format("avro")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/avro")
          )</code>
</details>
</p>

In [ ]:
# sind die Daten auf s3 angekommen
# <<HIER EIGENE CODE EINFÜGEN>>

In [ ]:
# Finde in der Darstellung der Datei die Metadaten und die eigentlichen Daten
# Da Avro ein Binärformat ist muss hier in cat die Flag auf True gesetzt werden
cat(bucket,"avro/part",True)

In [ ]:
read_avro=# HIER EIGENE CODE EINFÜGEN

read_avro.printSchema()
read_avro.show()

In [ ]:
# schreibe Datenset 3 (neue Spalte) in die gleiche Tabelle dazu (!! append NOT overwrite)

# HIER EIGENE CODE EINFÜGEN

<details>
<summary> &#8964 Lösung Datenset 3 als AVRO Datei anfügen</summary>
<p>
<code>write_avro=(df3
           .write
           .format("avro")
           .mode("append") # append
           .save(f"s3://{bucket}/avro")
          )</code>
</details>
</p>

In [ ]:
# alles nochmal einlesen und schauen ob die neue Spalte und die Schemas richtig erkannt wurden
# wiederholt sich langsam gell?

# <<HIER EIGENE CODE EINFÜGEN>>


<details>
<summary> &#8964 Lösung AVRO wieder einlesen</summary>
<p>
<code>
read_avro=spark.read.format("avro").load(f"s3://{bucket}/json")
read_avro.printSchema()
read_avro.show()
    </code>
</details>
</p>

In [ ]:
# Füge eine Zeile (df2) zu der AVRO Tabelle hinzu aber ändere den Datentyp für die id von long zu string
print("Schema vorher:")
df2.printSchema()


df2a=(df2
      # nur die Zeile Peter aus df2
      .where(f.col("account")=="peter")
      # ID als string statt als long
      .withColumn("id", f.col("id").cast("int"))
     )

print("Schema nachher:")
df2a.printSchema()


write_avro=(df2a
            .write
            .format("avro")
            .mode("append")
            .save(f"s3://{bucket}/avro")
           )


In [ ]:
# probiere das Verzeichnis mit verschiedenen Datentypen einzulesen
read_avro=(spark
           .read
           .format("avro")
           .load(f"s3://{bucket}/avro"))


read_avro.printSchema()
read_avro.show()

#### Erkenntnisse AVRO
* Werden Spaltennamen erhalten? 
* Gibt es ein Schema?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Avro Dateien bei der eine Spalte ein anderes Schema hat gelesen werden?

<hr style="height: 3px; background: gray;">

## Parquet

### Aufgabe:
Wiederhole die gleichen Schritte mit dem PARQUET Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als parquet schreiben (.format("parquet") und Pfad= .save(f"s3://{bucket}/parquet"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Metadaten in Datei identifizieren
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Schema Evolutiuon: Datenset 3 anfügen mit neuer Spalte anfügen
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert
6. Partion & Pushdown Filter: Execution Plan für verschiedene Filter anzeigen
6. Schema Enforcement: Datentyp in bestehender Spalte ändern und schauen ob und wie dies gehandhabt wird

In [ ]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_parquet=(df1
           .write
           # Fachliche Partitionierung beim Schreiben
           .partitionBy("account")
           .format("parquet")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/parquet")
          )


In [ ]:
# sind die Daten auf s3 angekommen
# <<HIER EIGENE CODE EINFÜGEN>>

In [ ]:
# schaue eine Datei im Detail an und finde die Metadaten
# <<HIER EIGENE CODE EINFÜGEN>>

<details>
<summary> &#8964 Lösung PARQUET Metadaten anzeigen</summary>
<p>
<code>
cat(bucket,"parquet/account=maria",True)
</code>
</details>
</p>

### Parquet: Filter Pushdown
Da das Parquet Format spalten basiert ist und für jede Spalte Metadaten vorhällt, können Programme die diese Dateien einlesen vor der Serialisierung (dem kompletten Einlesen in den Arbeitsspeicher) erst die Header scannen und entscheiden welche Dateien tatsächlich benötigt werden.  
Dies nennt man Attribut oder Filter Pushdown.  
Spark kann außerdem, wenn die Daten in Partition im Format `PartitionKey=value` diese automatisch erkennen und wenn ein Filter auf die Partition gelegt ist nur diesen Unterordner einlesen.  

**Aufgabe:** Untersuche den Execution Plan der Filter Operation 

In [ ]:
# Parquet Datei mit PartitionFilter laden
read_parquet=(spark
              .read.format("parquet")
              .load(f"s3://{bucket}/parquet")
              # Filter auf die Spalte über die partitioniert wurde
              .filter(f.col("account")=="alex")
             )

# Parquet mit normalem Filter laden
read_parquet2=(spark
              .read.format("parquet")
              .load(f"s3://{bucket}/parquet")
              # Filter auf die Spalte über eine normale Spalte
              .filter(f.col("balance")>1500)
             )

# Anzeigen des physischen Execution Plans um zu sehen welche Filter ins Dateisystem bzw. in die Parquet Datei gepusht werden
print("Partition Filter")
read_parquet.explain("simple")
print("Pushdow Filter")
read_parquet2.explain("simple")


#read_parquet.printSchema()
#read_parquet.show()

### Parquet: Schema Evolution
Schema Evolution ermöglicht es das Schema der Tabelle zu erweitern.  
Der Spark Parquet reader bietet verschiedenen Möglichkeiten mit Schemaerweiterungen umzugehen  
https://spark.apache.org/docs/latest/sql-data-sources-parquet.html#schema-merging

**Aufgabe:** Lese die Daten so ein, dass das Schema korrekt erweitert wird

In [ ]:
# Zeile mit neuer Spalte anfügen
write_parquet=(df3
           .write
           .format("parquet")
           .mode("append") # append
           # schreibe ohne zu Partitionieren direkt in ein neues Unterverzeichnis
           .save(f"s3://{bucket}/parquet/account=otto")
          )

In [ ]:
# einlesen mit der mergeSchema Option
read_parquet=(spark
              .read.format("parquet")
              # setzte die mergeSchema auf true/false um den Unterschied beim Einlesen zu sehen
              # Vegleiche: https://spark.apache.org/docs/latest/sql-data-sources-parquet.html#schema-merging
              #.option("mergeSchema", "false")
              .load(f"s3://{bucket}/parquet")
             )

read_parquet.printSchema()
read_parquet.show()

### Parquet: Schema Enforcement
Schema Enforcement sorgt dafür, dass in ein bestehendes Schema keine Daten mit falschen Typen geschrieben werden können

In [ ]:
# Datensatz mit falschem Datentyp anfügen
df2a=(df2.where(f.col("account")=="peter").withColumn("id", f.col("id").cast("string")))


# Zeile mit falschem Typ anfügen
write_parquet=(df2a
           .write
           .partitionBy("account")
           .format("parquet")
           .mode("append") # append
           .save(f"s3://{bucket}/parquet")
          )

In [ ]:
# einlesen mit der mergeSchema Option
read_parquet=(spark
              .read.format("parquet")
              # setzte die mergeSchema auf true/false um den Unterschied beim Einlesen zu sehen
              .option("mergeSchema", "false")
              .load(f"s3://{bucket}/parquet")
             )

read_parquet.printSchema()
read_parquet.show()

#### Erkenntnisse Parquet
* Sind Parquet Dateien selbsterklärend (haben ein Spalten und Typenschema )
* Partitioning and Partion Discovery: werden die Daten in Verzeichnisse geschriebe und wieder als Partitionen erkannt?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Parquet Dateien bei der eine Spalte ein anderes Schema hat gelesen werden?

<hr style="height: 3px; background: gray;">

# Delta
Das Delta Format fügt Parquet Dateien einen zusätzlichen Layer an Metadatan hinzu und erfüllt mit dem entsprechenden Treiber alle ACID Eigenschaften einer Datenbank und mehr. 

A = **Atomic** heißt alle Datenänderungen werden wie eine einzige Operation verarbeitet. Dies bedeutet, dass entweder alle Änderungen durchgeführt werden oder keine. Wenn das schreiben also mitten drin Fehlschlägt werden alle Daten dieser Schreiboperation die bereits geschrieben wurden wieder entfernt, bzw. nicht als erfolgreich geschrieben markiert.  
T = **Consistency** bedeutet, wenn eine Transaktion beginnt und wenn eine Transaktion endet, befinden sich die Daten in einem konsistenten Zustand.  
I = **Isolation** und bedeutet, dass der Übergangszustand einer Transaktion für andere Transaktionen nicht sichtbar ist. Dies führt dazu, dass Transaktionen, die gleichzeitig ablaufen, sich nicht gegenseitig beeinflussen oder blockieren.  
D =**Durability** heißt, das die Datenänderungen nach erfolgreich abgeschlossener Transaktion erhalten bleiben und werden nicht rückgängig gemacht werden, selbst wenn ein Systemausfall auftritt.  

**Time Travel** bei Delta bedeutet, dass jede Datenänderung als eigene Version aufgezeichnet wird und jederzeit zu einer alten Version zurück gegegangen werden kann.

### Aufgabe:
Wiederhole die gleichen Schritte mit dem DELTA Format und schaue wie sich hier Schema und neue Spalten verhalten

1. Datenset 1 als DELTA schreiben (.format("delta") und Pfad= .save(f"s3://{bucket}/delta"))
2. Dateien und Inhalt anzeigen, vestehen was da passiert ist
3. Metadaten und Deltalog in Datei verstehen
3. Daten wieder einlese und checken ob es ein Schema und Spaltennamen gibt
4. Schema Evolutiuon: Datenset 3 anfügen mit neuer Spalte anfügen
5. Daten wieder einlesen und checken was mit der neuen Spalte passiert
6. Partion & Pushdown Filter: Execution Plan für verschiedene Filter anzeigen
6. Schema Enforcement: Datentyp in bestehender Spalte ändern und schauen ob und wie dies gehandhabt wird

In [ ]:
# Schreibe die Daten df1 als Delta Datei in den Pfad bucket/delta
write_delta=(df1
           .write
           .format("delta")
           #.option("mergeSchema", "false")
           .mode("overwrite") 
           .save(f"s3://{bucket}/delta")
          )

In [ ]:
ls(bucket,"delta")

In [ ]:
ls(bucket,"delta/_delta_log")

In [ ]:
# untersuche die Dateien des Deltalogs und versuche zu verstehen wie die Versionierung und Schema Validierung damit funktioniert
cat(bucket,"delta/_delta_log")

In [ ]:
# lese die gerade geschriebenen Delta Tabelle wieder ein und zeige sie an, überprüfe das Schema
# <<HIER EIGENE CODE EINFÜGEN>>


<details>
<summary> &#8964 Lösung DELTA einlesen</summary>
<p>
<code>
read_delta=spark.read.format("delta").load(f"s3://{bucket}/delta")
read_delta.show()
</code>
</details>
</p>

### Delta: Schema Evolution
**Aufgabe:** Verstehe die Option *mergeSchema* on write

In [ ]:
# Zeile mit zusätzlicher Spalte anfügen (df3)
write_delta=(df3
           .write
           .format("delta")
           # Bei Delta kann bein Schreiben gesetzt werden ob die Tabelle erweitert werden soll oder nicht, Default ist false. 
           # Führe den Code zuerst ohne diese Option aus und schaue das Ergebnis, an 
           #.option("mergeSchema", "false")
           .mode("append") # append
           .save(f"s3://{bucket}/delta")
          )


# überprüfe ob die neue Spalte korrekt angefügt wurde
read_delta=spark.read.format("delta").load(f"s3://{bucket}/delta")
read_delta.show()

In [ ]:
cat(bucket,"delta/_delta_log")

### Delta: Schema Enforcement
Schema Enforcement bedeutet soll garantieren, dass keine Daten mit falschen Datentyp der Tabelle abgefügt werden  

**Aufgabe:** verstehe die Option *mergeSchema* im Kontext von Datentyp Änderungen bei bestehenden Spalten

In [ ]:
# füge eine Zeile mit falschen Datetyp für eine bestehenden Spalte an
write_delta=(df2
           # Eine Zeile aus dem df2 filtern
           .where(f.col("account")=="peter")
           # Bestehenden Spaltentyp ändern
           .withColumn("id", f.col("id").cast("string"))
           .write
           .format("delta")
           # Bei Delta kann bein Schreiben gesetzt werden ob die Tabelle erweitert werden kann oder nicht, Default ist false
           #.option("mergeSchema", "false")
           .mode("overwrite") # append
           .save(f"s3://{bucket}/delta")
          )

### Delta: Schema Replacement
Delta bieten die Möglichkeit das Schema einer Tabelle zu ändern, also z.B. eine bestehende Spalte umzubenennen und deren Datentyp zu ändern.   
Dokumentation mit Beispielen: https://docs.delta.io/latest/delta-batch.html#replace-table-schema  

**Aufgabe:** Ändere in der bestehenden Tabelle den Datentyp der Spalte `id` auf `string` und den Spaltennamen `new` zu `comment`. 

In [ ]:
# Update das Schema der bestehenen Delta Dateien

### Delta: History und Metadaten
Im Delta Log werde alle Transaktionen mit zahlreichen Metadaten gespeichert. Das Spark Modul, der Treiber, um diese Daten auszulesen bietet zahlreiche Möglichkeiten diese Daten zu analysieren

**Aufgabe:** Lese den History Log ein und verstehe was in den einzelnen Attributen steht. Treffe eine Auswahl der interessanten Informationen

In [ ]:
# Erzeuge ein DeltaTable Objekt was alle Zusatzeigenschaften von Delta bereitstellt
deltaTable = DeltaTable.forPath(spark, f"s3://{bucket}/delta")

# Historie aus den Delta Logs erzeugen
fullHistoryDF = deltaTable.history() 

# Alle verfügbaren Spalten anzeigen
fullHistoryDF.printSchema()

In [ ]:
# Wähle die wichtigen Felder aus der Historie aus und zeige sie an
# fullHistoryDF.select(<<CODE EINFÜGEN>>).show(truncate=True)

# Löse die genested Spalten in eigene Spalten auf und zeige folgende Attribute in eigenen Spalten `mode`, `numFiles` und `numOutputRows` 
#fullHistoryDF.select(<<CODE EINFÜGEN>>).show()

<details>
<summary> &#8964 Lösung Historie analysieren</summary>
<p>
<code>
deltaTable = DeltaTable.forPath(spark, f"s3://{bucket}/delta")
fullHistoryDF = deltaTable.history() 
fullHistoryDF.select("version","readVersion","timestamp","userId","operation","operationParameters","operationMetrics","userMetadata").show(truncate=False)

# Zugriff auf genested struc Objecte mit `spalte.attributname`
fullHistoryDF.select("version","readVersion","timestamp","operation","operationParameters.mode","operationMetrics.numFiles","operationMetrics.numOutputRows").show(truncate=False)
read_json.printSchema()
read_json.show()
    </code>
</details>
</p>

In [ ]:
# Erkunde die Metadaten aus der Funktion deltaTable.detail()

#<<CODE EINFÜGEN>>

<details>
<summary> &#8964 Lösung Details anezigen</summary>
<p>
<code>
deltaTable = DeltaTable.forPath(spark, f"s3://{bucket}/delta")
detailsDF = deltaTable.detail()
detailsDF.show()
detailsDF.select("location","numFiles","tableFeatures").show(truncate=False)
</code>
</details>
</p>

### Delta: Time Travel
Die Time Travel Funktion von Delta ermöglicht es den Zustand der Datentabelle zu einem bestimmten Zeitpunkt in der Vergangenheit wiederherzustellen oder Änderungen zu verfolgen.   
Time Travel ermöglicht es, vorherige Versionen der Daten abzufragen und historische Analysen durchzuführen, ohne auf separate Backups oder Snapshots angewiesen zu sein.

**Aufgabe:** lese verschiedenen Datenstände nach Versionsnummer oder Timestap ein.  
Weitere Informationen hierzu finden sich in https://delta.io/blog/2023-02-01-delta-lake-time-travel/

In [ ]:
spark.read.format("delta").option(<<CODE EINFÜGEN>>).load(f"s3://{bucket}/delta").show()


### Delta: Merge (Upsert)
Delta bietet die Möglichkeiten auf bestehenden Dateien ein Daten Upsert durchzuführen.   
Ubsert oder Merge bedeutet zu prüfen ob es die neue Zeile für einen bestimmten Schlüssel schon in den Daten gibt und wenn ja diese zu updaten und wenn nein sie neu hinzuzufügen

**Aufgabe:** Merge den Datensatz df2 mit dem der Änderung der `balance` für `Alex` und überprüfe ob das Update korrekt war

In [ ]:
deltaTable2 = DeltaTable.forPath(spark, f"s3://{bucket}/delta")

# Spalte anfügen, da merge nur funktioniert wenn das Schema stimmt
df2a=df2.withColumn("new",f.lit("test"))

print("++ Datensatz der auf bestehende Daten upserted/merged werden soll")
df2a.show()
print("++ Bestehender Datensatz auf s3")
deltaTable2.toDF().show()

In [ ]:
# Verwendung der merge Funktion (es gibt auch eine update() oder delete() Funktion)
dt3=(deltaTable2.alias("oldData")
      .merge(df2a.alias("newData"),
            "oldData.account = newData.account AND oldData.dt_transaction = newData.dt_transaction")
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
      .execute()
    )

deltaTable2.toDF().show()

### Delta: Roleback
Delta bietet die Möglichkeiten direkt auf den Datenstand einer bestimmten Version oder eines Zeitpunktes zurück zu gehen

In [ ]:
# Verwende die DeltaTable Funktion restoreToVersion(0) oder restoreToTimestamp("yyyy-mm-dd")

<details>
<summary> &#8964 Hilfreiche Befehle</summary>
<p>
<code>
deltaTable2.toDF().show()

deltaTable2.restoreToVersion(0)
#restoreToTimestamp
#isDeltaTable
deltaTable2.toDF().show()
spark.read.format("delta").load(f"s3://{bucket}/delta").show()

</code>
</details>
</p>

#### Erkenntnisse Delta
* Wie funktioniert das Metadatenmanagement und was steht im Delta Log?
* Schema Evolution: Kann das Schema erweitert werden, also eine neue Spalte angefügt werden?
* Schema Enforcement on write: Kann eine Spalte mit falschem Datetyp einfach beim schreiben hinzugefügt werden? 
* Schema Enforcement on read: Kann ein Verzeichnis mit mehreren Parquet Dateien bei der eine Spalte ein anderes Schema hat gelesen werden? (um die Ecke Denk Frage)
* Was ermöglichen mir die Metadaten (Historie, Audit etc)
* Was ist der Vorteil der Merge Funktion? Wie müsste ich sonst Dateibasiert einen Merge/Update durchführen?


<hr style="height: 3px; background: gray;">

## Iceberg
Das Iceberg Format Ist vergleichbar mit dem Delta Format Parquet, Avro oder ORC Dateien einen zusätzlichen Layer an Metadatan hinzu und erfüllt mit dem entsprechenden Treiber alle ACID Eigenschaften einer Datenbank.   
**Unterschied zu Delta:**  
* Das Delta Format verwendet ein Transaktionsprotokoll, um inkrementelle Updates zu erfassen und eine vollständige Versionierung der Daten zu ermöglichen. Dies bedeutet, dass Delta Änderungen an den Daten in einem einzelnen Parquet-Dateisystem erfasst. Delta ist eng mit Spark und der Delta Lake Plattform von Databricks verbunden und verwedet als darunterliegendes Datenformat immer Parqeut Dateien
* Iceberg hat ebenfalls eine append-only-Architektur, bei der Daten in sogenannten "Snapshot-Dateien" gespeichert werden. Iceberg ist älter und unabhängig von einer speziellen Plattform. Daher es gibt wesentlich mehr Big Data Engines die Iceberg voll unterstützen (Spark, Hive, Trino, Drill, Presto).

**Naming Unterschiede:**  
Bei Delta spricht man von `Versionen` und `Time Travel`. 
Bei Iceberg heißt dies `Snapshots` und `Snapshot roleback`  

Eine weitere Besonderheit bei Iceberg ist, dass es für Daten die Möglichkeiten von Branching und Tagging gibt. Es können also Variationen der Daten in einem z.B. Development Branch gepflegt und bearbeitet weiter entwickelt werden (https://iceberg.apache.org/docs/latest/branching/)

Im Unterchied zu Delta benötigt Spark zum arbeiten mit Iceberg immer einen Catalog oder Metastore um Details über die Dateien abzulegen.

### Iceberg: Catalog und Metastore

In [5]:
# Anzeigen der in Spark eingebundenen Metadaten Cataloge
# Diese Spark Session wurde mit einer Anbindung an den Hive Metastore gestartet
## Konfiguration aus für die Spark Session erzeugt einen Catalog ice vom Typ hive
## conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
## conf.set("spark.sql.catalog.ice","org.apache.iceberg.spark.SparkCatalog") 
## conf.set("spark.sql.catalog.ice.type","hive") 
## conf.set("spark.sql.catalog.ice.uri","thrift://hive-metastore.hive.svc.cluster.local:9083") 


spark.sql("SHOW catalogs").show()
spark.sql("SHOW databases from ice").show()
spark.sql("show tables from ice.iceberg").show()

+-------------+
|      catalog|
+-------------+
|          ice|
|spark_catalog|
+-------------+

+---------+
|namespace|
+---------+
|  default|
|  iceberg|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  iceberg|  iceberg|      false|
+---------+---------+-----------+



In [ ]:
# Zunächst muss in dem Ice Catalog eine Datenbankabstraktion angelegt werden mit dem s3 Pfad wo die Daten zu dieser Tabellen liegen
# create a Database(name=<db_name>, locationUri='s3a://<bucket>/')
spark.sql(f"CREATE DATABASE IF NOT EXISTS ice.iceberg LOCATION 's3a://{bucket}/'")

In [ ]:
# Jetzt erscheint diese Datenbank im Iceberg Catalog
spark.sql("SHOW databases from ice").show()

In [ ]:
# Es gibt aber noch keine Tabellen in dieser Datenbankabstraktion
spark.sql("show tables from ice.iceberg").show()

In [ ]:
# Die Tabelle kann ach wieder aus dem Hive Metastore, aus dem Catalog gelöscht werden
#spark.sql("drop table iceberg.iceberg")

In [9]:
# Um Daten unter Verwendung des Catalogs nach s3 zu schreiben wird nicht der direkte Pfad verwendet sondern die Datenbank Referenz wo der Pfad hinterlegt ist
# Die Datenbank iceberg -> s3://bucket/
# die Tabelle soll jetzt nach ->s3://bucket/iceberg
# geschrieben wird mit der Methode `saveAsTable("datenbank.tabelle")` und der Tabellennamen wir dann als Prefix auf s3 verwendet
write_iceberg=(df1
                  .write
                  .format("iceberg")
                  .mode("overwrite")
                  .saveAsTable("ice.iceberg.iceberg")
               )


In [ ]:
# Versuche auf dem Dateisystem mit ls und cat die Struktur der Metadaten zu verstehen
# Dieser Artikel kann dabei helfen https://medium.com/snowflake/understanding-iceberg-table-metadata-b1209fbcc7c3
ls(bucket,"iceberg")

In [ ]:
# zeige die Daten wieder an
iceberg_df = spark.read.table("ice.iceberg.iceberg")
iceberg_df.show()

# Da es sich bereits um eine Tabellenabstraktion handelt, kann auch direkt mit Spark SQL gelesen werden
spark.sql("SELECT * FROM ice.iceberg.iceberg;").show()


### Iceberg: Schema Evolution
**Aufgabe:** Verstehe wie eine Spaltenerweiterung bei Iceberg gemacht wird

In [12]:
# Füge Datensatz 2 mit einer zusätzlichen Spalte hinzu
write_iceberg=(df3
               .write
               .format("iceberg")
               #.option("mergeSchema","true")
               .mode("append")
               .saveAsTable("ice.iceberg.iceberg")
               )

In [11]:
# Um eine neue Spalte hinzuzufügen muss das Schema wie bei einer SQL Tabelle zuerst geändert werden. Es gibt keine automatische Anpassung wie bei Delta via die Option mergeSchema
# Ändere das Schema mit folgendem Befehl und führe dann obige Schreiboperation nochmal durch
spark.sql("ALTER TABLE ice.iceberg.iceberg ADD COLUMNS (new VARCHAR(50))")

DataFrame[]

In [13]:
spark.sql("SELECT * FROM ice.iceberg.iceberg;").show()

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
|  1|   alex|    2019-01-01|   1000|         null|
|  2|   alex|    2019-02-01|   1500|         null|
|  4|  maria|    2020-01-01|   5000|         null|
|  3|   alex|    2019-03-01|   1700|         null|
+---+-------+--------------+-------+-------------+



### Iceberg: Schema Enforcement

**Aufgabe:** verstehe wie Schema Enforcement bei Iceberg funktioniert. Korrigiere den Code bis die Zeile korrekt angefügt werden kann

In [ ]:
# füge eine Zeile mit falschen Datetyp für eine bestehenden Spalte an
try:
    write_iceberg=(df2
           # Eine Zeile aus dem df2 filtern
           .where(f.col("account")=="peter")
           # Bestehenden Spaltentyp ändern
           .withColumn("id", f.col("id").cast("string"))
           #.withColumn("new", f.lit("peter").cast("string"))
           .write
           .format("iceberg")
           .mode("append") # append
           .saveAsTable("ice.iceberg.iceberg")
          )
    print("Zeile angefügt")
except Exception as error:
    print("ERROR Writing to Iceberg")
    print(error)

In Iceberg kann der Typ einer bestehenden Tabelle nicht geändert werden.   
Einzige Möglichkeit ist es die Tabelle einzulesen, neu zu casten und in einen neue Tabelle zu schreiben.   
Diese kann dann anschließend wieder umbenannt werden auf den alten Namen

### Iceberg: History und Metadaten
Iceberg bietet zahlreiche Funktionen um die verschiedenen Metadaten auszulesen.   
Die Funktionen werden immer an die Tabelle angefügt und funktionieren sowohl in der Dataframe als auch in der SQL Syntax
```
spark.read.table("ice.iceberg.iceberg.history").show()
spark.sql("SELECT * FROM ice.iceberg.iceberg.history;").show()
```

**Aufgabe:** Untersuche und verstehe die Metadaten folgender Methoden:  
`history`, `files`, `snapshots`, `manifests`, `partitions`

Weitere Informationen zu den Metadaten und dem Zugrif darauf finden sich in der Dokumentation
https://iceberg.apache.org/docs/latest/spark-queries/

In [14]:
# << EIGENEN CODE EINFÜGEN >>
spark.read.table("ice.iceberg.iceberg.history").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2023-06-29 15:19:...|4422470565428377895|               null|               true|
|2023-06-29 15:19:...|7483311904055840977|4422470565428377895|               true|
+--------------------+-------------------+-------------------+-------------------+



<details>
<summary> &#8964 Lösung Historie analysieren</summary>
<p>
<code>
spark.sql("SELECT * FROM iceberg.iceberg.history;").show()
spark.sql("SELECT * FROM iceberg.iceberg.files;").show()
spark.sql("SELECT * FROM iceberg.iceberg.snapshots;").show()

## alternative syntax example:
spark.read.format("iceberg").load("iceberg.iceberg.files").show()
</code>
</details>
</p>



### Iceberg: Iceberg: Time Travel
Bei Iceberg gibt es statt Versionen Snapshots und einige fein granularere Möglichkeiten in der Daten Historie zurück zu gehen.
z.B. können alle Time Travel Operationne auch auf die Metadaten angewendet werden. Es kann also nachgeschaut werden wie die Metadaten zu einem bestimmten Zeitpunkt/Snapshot ausgesehen haben.

- ```snapshot-id``` selects a specific table snapshot
- ```as-of-timestamp``` selects the current snapshot at a timestamp, in milliseconds
- ```branch``` selects the head snapshot of the specified branch. Note that currently branch cannot be combined with as-of-timestamp.
- ```tag``` selects the snapshot associated with the specified tag. Tags cannot be combined with as-of-timestamp.

In [17]:
# from the results of iceberg_table.snapshots get the snapshots IDs
snapshot1 = spark.read \
                 .option("snapshot-id", "<<Valide Snapshot ID eintragen>>") \
                 .format("iceberg") \
                 .load("ice.iceberg.iceberg").show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   alex|    2019-01-01|   1000|
|  2|   alex|    2019-02-01|   1500|
|  4|  maria|    2020-01-01|   5000|
|  3|   alex|    2019-03-01|   1700|
+---+-------+--------------+-------+



In [ ]:
snapshot2 = spark.read \
                 .option("snapshot-id", "<<Valide Snapshot ID eintragen>>") \
                 .format("iceberg") \
                 .load("ice.iceberg.iceberg").show()

<hr style="height: 3px; background: gray;">

In [ ]:
spark.stop()